# NLP 3A - Kaggle Competition

## 0 - Imports

In [1]:
import json

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import string
import nltk
from sklearn.utils import shuffle
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler

stop_words = stopwords.words('english')

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import matplotlib.pyplot as plt
import seaborn as sns

from transformers import BertTokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## I - Baseline code

In [2]:
with open('test.json', 'r') as file:
    data = json.load(file)

f_data = {}
for k, v in data.items():
    f_data[k] = v[:95] # Ok, en gros f_date = {'Sport': [Liste des sequences de theme sport],...}

processed_data = {
    'text': [], 'target': []
}
for k, v in f_data.items():
    processed_data['text'] += v
    processed_data['target'] += [k] * len(v) # {'text': [Liste des phrases], 'target': [Liste des labels]}

df_test = pd.DataFrame.from_dict(processed_data)
df_test.head()

,text,target
0,The team scored a last-minute goal to win the ...,Sports
1,The tennis player served an ace to win the match.,Sports
2,"The basketball game went into overtime, but th...",Sports
3,The marathon runner broke the world record wit...,Sports
4,The soccer match ended in a draw after a thril...,Sports


In [3]:
with open('cleaned_generated_train.json', 'r') as file:
    data = json.load(file)

f_data = {}
for k, v in data.items():
    f_data[k] = v[:95] # Ok, en gros f_date = {'Sport': [Liste des sequences de theme sport],...}

processed_data = {
    'text': [], 'target': []
}
for k, v in f_data.items():
    processed_data['text'] += v
    processed_data['target'] += [k] * len(v) # {'text': [Liste des phrases], 'target': [Liste des labels]}

df = pd.DataFrame.from_dict(processed_data)
df["text"] = df["text"].str.replace(r"\\u[0-9A-Fa-f]{4}", "", regex=True)  # Remove Unicode escape sequences
df["text"] = df["text"].str.replace(r"[^\x00-\x7F]+", "", regex=True)  # Remove non-ASCII characters
df["text"] = df["text"].str.strip()  # Remove leading and trailing whitespace
df.head()

,text,target
0,,Politics
1,We'll send it along in case you find yourself ...,Politics
2,"The phrase should be less than one word, but i...",Politics
3,The task will include writing an article that ...,Politics
4,"See, this one's pretty cool because it comes f...",Politics


In [4]:
# with open('train.json', 'r') as file:
#     data = json.load(file)

# f_data = {}
# for k, v in data.items():
#     f_data[k] = v[:95] # Ok, en gros f_date = {'Sport': [Liste des sequences de theme sport],...}

# processed_data = {
#     'text': [], 'target': []
# }
# for k, v in f_data.items():
#     processed_data['text'] += v
#     processed_data['target'] += [k] * len(v) # {'text': [Liste des phrases], 'target': [Liste des labels]}

# df = pd.DataFrame.from_dict(processed_data)
# df.head()

In [5]:
# X_train, X_test, y_train, y_test = train_test_split(df['text'].values, df['target'].values, test_size=0,
#                                                    random_state=123, stratify=df['target'].values)

X_train = df['text'].values
y_train = df['target'].values
X_train, y_train = shuffle(X_train, y_train)
with open('test_shuffle.txt', 'w') as file:
    for item in X_train.tolist():
        file.write(item + "\n")

with open('y_test_shuffle.txt', 'w') as file:
    for item in y_train.tolist():
        file.write(item + "\n")

y_test = df_test['target'].values
X_test = df_test['text'].values

# II - Models

### 1 - TfidfVectorizer + Random Forest

In [6]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_train_vectors = tfidf_vectorizer.fit_transform(X_train)
tfidf_test_vectors = tfidf_vectorizer.transform(X_test)

classifier = RandomForestClassifier() # RF baseline model
classifier.fit(tfidf_train_vectors, y_train)

y_pred = classifier.predict(tfidf_test_vectors)


with open('y_pred_shuffle.txt', 'w') as file:
    for item in y_pred.tolist():
        file.write(item + "\n")
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
# print(classification_report(y_test, y_pred))

Accuracy: 0.11


In [7]:
results_df = pd.DataFrame({
    'Sentence': X_test,
    'Actual Target': y_test,
    'Predicted Target': y_pred
})

### 2 - Bert

In [8]:
num_unique_labels = df['target'].nunique()
print(f'The number of unique labels is: {num_unique_labels}')

The number of unique labels is: 12


In [9]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=12).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

class SentenceDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        sentence = self.sentences[index]
        label = self.labels[index]
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [11]:
BATCH_SIZE = 16
MAX_LEN = 128
EPOCHS = 4
LEARNING_RATE = 2e-5

train_dataset = SentenceDataset(X_train, y_train_encoded, tokenizer, MAX_LEN)
test_dataset = SentenceDataset(X_test, y_test_encoded, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
def train_epoch(model, data_loader, optimizer, device):
    model.train()
    losses = []
    correct_predictions = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return correct_predictions.double() / len(data_loader.dataset), torch.mean(torch.tensor(losses))

def eval_model(model, data_loader, device):
    model.eval()
    correct_predictions = 0
    losses = []
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return correct_predictions.double() / len(data_loader.dataset), torch.mean(torch.tensor(losses)), all_preds, all_labels

In [13]:
for epoch in range(EPOCHS):
    train_acc, train_loss = train_epoch(model, train_loader, optimizer, device)
    val_acc, val_loss, y_pred, y_test = eval_model(model, test_loader, device)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:2834: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-pa

In [14]:
results_df_bert = pd.DataFrame({
    'Sentence': X_test,
    'Actual Target': y_test,
    'Predicted Target': y_pred
})
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.7035087719298245

### 3 - Bert + Logistics

In [86]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

def bert_tokenize_texts(texts, tokenizer):
    return [" ".join(tokenizer.tokenize(text)) for text in texts]

X_train = bert_tokenize_texts(df['text'], tokenizer)
X_test_tokenized = bert_tokenize_texts(X_test, tokenizer)

vectorizer = TfidfVectorizer(max_features=5000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test_tokenized)

logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train_vectorized, y_train)

y_pred = logreg.predict(X_test_vectorized)

In [87]:
results_df_logreg = pd.DataFrame({
    'Sentence': X_test,
    'Actual Target': y_test,
    'Predicted Target': y_pred
})

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print(results_df_logreg)

Accuracy: 0.0


ValueError: Mix of label input types (string and number)

# III - Results

In [52]:
results_df_bert.head(5)

,Sentence,Actual Target,Predicted Target
0,The team scored a last-minute goal to win the ...,9,8
1,The tennis player served an ace to win the match.,9,8
2,"The basketball game went into overtime, but th...",9,7
3,The marathon runner broke the world record wit...,9,8
4,The soccer match ended in a draw after a thril...,9,8


In [53]:
results_df.head(5)

,Sentence,Actual Target,Predicted Target
0,The team scored a last-minute goal to win the ...,Sports,Fashion
1,The tennis player served an ace to win the match.,Sports,Fashion
2,"The basketball game went into overtime, but th...",Sports,Technology
3,The marathon runner broke the world record wit...,Sports,Environment
4,The soccer match ended in a draw after a thril...,Sports,Technology


# II - Label to submission

In [15]:
with open('y_pred_shuffle.txt', 'r') as file:
    datas = file.readlines()

with open('y_test_shuffle_for_kaggle.txt', 'w') as file:
    i = 0
    file.write(f"ID,Usage,Label\n")
    for data in datas:
        file.write(f"{i},Private,{data}") if i % 2 == 1 else file.write(f"{i},Public,{data}")
        i += 1

with open('y_test_shuffle.txt', 'r') as file:
    datas = file.readlines()

with open('y_test_shuffle_for_kaggle.txt', 'w') as file:
    i = 0
    file.write(f"ID,Usage,Label\n")
    for data in datas:
        file.write(f"{i},Private,{data}")
        i += 1